In [ ]:
import pandas as pd

df_fil = pd.read_csv('filtered_news.csv')
df_fil

,date,region,text
0,2022-01-01,belgorod,Вячеслав Гладков: к строительству детских площ...
1,2022-01-01,belgorod,В правительстве Белгородской области оценили э...
2,2022-01-01,belgorod,В 2022 году поддержку по соцконтракту получат ...
3,2022-01-01,belgorod,Уважаемые жители села Шеино!
4,2022-01-01,belgorod,Уважаемые жители села Шеино!
...,...,...,...
544278,2022-12-31,grozniy,Рамзан Кадыров об обстановке в Грозном: Все ск...
544279,2022-12-31,grozniy,Молодогвардейцы проверили цены на проживание в...
544280,2022-12-31,grozniy,Уходящий год для Кавказа стал Годом сплочения ...
544281,2022-12-31,grozniy,В Минобрнауки ЧР подвели итоги освещения нацпр...


## Доп. очистка

In [ ]:
df_fil = df_fil.drop_duplicates(subset=["text"])
df_fil.info()

<class 'pandas.core.frame.DataFrame'>
Index: 481197 entries, 0 to 544282
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    481197 non-null  object
 1   region  481197 non-null  object
 2   text    481197 non-null  object
dtypes: object(3)
memory usage: 14.7+ MB


In [ ]:
def word_count(text: str) -> int:
    return len(text.split())

df_fil = df_fil[df_fil["text"].apply(word_count) >= 5]

In [ ]:
import re

BAD_PATTERNS = [
    r"с наступающ",
    r"поздравля",
    r"важная информация",
    r"объявлени",
    r"график работы",
    r"режим работы",
]

bad_regex = re.compile("|".join(BAD_PATTERNS), re.IGNORECASE)

def is_noise(text):
    return bool(bad_regex.search(text))

In [ ]:
df_fil = df_fil[~df_fil["text"].apply(is_noise)]
df_fil.info()

<class 'pandas.core.frame.DataFrame'>
Index: 452126 entries, 0 to 544282
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    452126 non-null  object
 1   region  452126 non-null  object
 2   text    452126 non-null  object
dtypes: object(3)
memory usage: 13.8+ MB


# BERTopic

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
embedding_model = SentenceTransformer(
    "ai-forever/FRIDA",
    device=device
    )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from umap import UMAP

umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    low_memory=True,
    random_state=42
)

In [ ]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(
    min_cluster_size=50,
    min_samples=10,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True
)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),
    min_df=10,
    max_df=0.95,
    stop_words=None
)

In [ ]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 14.0 MB/s eta 0:00:00


In [ ]:
from bertopic.representation import KeyBERTInspired

representation_model = KeyBERTInspired()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(
    stop_words=custom_ru_stopwords,
    token_pattern=r"(?u)\b[а-яА-ЯёЁ]{3,}\b",
    min_df=10
)

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(
    low_memory=True,
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
    calculate_probabilities=False,
    verbose=True
)

In [ ]:
texts = df_fil["text"].tolist()

In [ ]:
embeddings = embedding_model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=True,
    prompt_name="categorize_topic"
)

import numpy as np
np.save("embeddings.npy", embeddings.astype("float32"))

In [ ]:
embeddings = np.load("embeddings.npy")

In [ ]:
topics, probs = topic_model.fit_transform(
    texts,
    embeddings=embeddings
    )

2025-12-18 13:48:26,818 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-18 14:06:09,457 - BERTopic - Dimensionality - Completed ✓
2025-12-18 14:06:09,472 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-18 14:07:27,414 - BERTopic - Cluster - Completed ✓
2025-12-18 14:07:27,535 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-18 14:08:37,156 - BERTopic - Representation - Completed ✓


In [ ]:
df_fil["topic"] = topics
df_fil.to_csv('allFilteredNewsTopics.csv')

In [ ]:
df_fil

,date,region,text,topic
0,2022-01-01,belgorod,Вячеслав Гладков: к строительству детских площ...,-1
1,2022-01-01,belgorod,В правительстве Белгородской области оценили э...,0
2,2022-01-01,belgorod,В 2022 году поддержку по соцконтракту получат ...,-1
5,2022-01-01,belgorod,Белгородэнерго напомнило о правилах электробез...,814
6,2022-01-01,belgorod,В Белгородской области отремонтируют 110 км до...,361
...,...,...,...,...
544278,2022-12-31,grozniy,Рамзан Кадыров об обстановке в Грозном: Все ск...,-1
544279,2022-12-31,grozniy,Молодогвардейцы проверили цены на проживание в...,659
544280,2022-12-31,grozniy,Уходящий год для Кавказа стал Годом сплочения ...,531
544281,2022-12-31,grozniy,В Минобрнауки ЧР подвели итоги освещения нацпр...,319


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,208738,-1_области_районе_2022 года_2022,"[области, районе, 2022 года, 2022, область, ра...",[В 2022 году продолжается реализация мероприят...
1,0,4596,0_местных инициатив_области 2021_приоритет 203...,"[местных инициатив, области 2021, приоритет 20...",[Заслушивание Минобра ЗК об итогах работы за 2...
2,1,3300,1_развитие инфраструктуры_2022 году_инфраструк...,"[развитие инфраструктуры, 2022 году, инфрастру...",[В Кировской области на реализацию антикризисн...
3,2,3188,2_обсудили меры_рассказали мерах_пакет мер_по ...,"[обсудили меры, рассказали мерах, пакет мер, п...","[Поддержка малого и среднего бизнеса, Опрос су..."
4,3,2595,3_на развитие_хозяйствах_сельское_сельском,"[на развитие, хозяйствах, сельское, сельском, ...","[Игорь Гордеев: ""Для сельского хозяйства сегод..."
...,...,...,...,...,...
1014,1013,50,1013_мероприятия направленные_продолжается про...,"[мероприятия направленные, продолжается проект...",[В Ижемском районе обстановка по линии незакон...
1015,1014,50,1014_на строительство_объектов инфраструктуры_...,"[на строительство, объектов инфраструктуры, пр...",[В Ингушетии потратят более 400 млн руб. на мо...
1016,1015,50,1015_получат жители_тульская область_область п...,"[получат жители, тульская область, область пол...",[Дополнительные средства на переселение гражда...
1017,1016,50,1016_областном фестивале_региональный фестивал...,"[областном фестивале, региональный фестиваль, ...",[Фестиваль творчества среди детей с ограниченн...


In [ ]:
topic_model.get_topic_info().describe()

,Topic,Count
count,1019.000000,1019.000000
mean,508.000000,443.695780
std,294.304264,6542.033911
min,-1.000000,50.000000
25%,253.500000,74.000000
50%,508.000000,120.000000
75%,762.500000,238.000000
max,1017.000000,208738.000000


In [ ]:
topic_model.save(
    "bertopic_regions",
    serialization="safetensors"
)